# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
2. 워드 임베딩 모델을 학습시킨다.
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다.
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다.

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.
ex) A: 산, B: 바다, C: 나무, D: 물

**<출력 예시>**

관계 [수긍: 추락 = 대사관 : ?]

모델이 예측한 가장 적합한 단어: 잠입

당신의 답변과 모델 예측의 유사도: 0.34

아쉽네요. 더 생각해보세요.

In [1]:
from pathlib import Path

# ▶ 파일 경로 (필요시 수정)
STOPWORDS_PATH = Path("ko_stopwords.txt")
TRANSCRIPT_PATH = Path("transcript.v.1.4.txt")
EMB_PATH = Path("cc.ko.300.vec")

# ▶ 옵션
USE_EMBEDDING_FILTER = True   # 모델 사전에 있는 단어만 남기려면 True
SHOW_ROWS = 5                  # 미리보기 개수
MIN_TOKEN_LEN = 2              # 1글자 토큰 제거
ALLOW_POS = ("N",)  # 명사/동사/형용사만
MIN_FREQ = 1                   # 후보 단어 풀 최소 빈도

In [2]:
import re, unicodedata
from collections import Counter
from typing import List, Set, Iterable, Optional, Tuple, Dict
from konlpy.tag import Okt

okt = Okt()

RE_URL     = re.compile(r"https?://\S+|www\.\S+")
RE_EMAIL   = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
RE_HTML    = re.compile(r"<[^>]+>")
RE_EMOJI   = re.compile("[\U00010000-\U0010FFFF]", flags=re.UNICODE)
RE_HANJA = re.compile(r"[\u3400-\u4DBF\u4E00-\u9FFF\uF900-\uFAFF]")
RE_ETC     = re.compile(r"[^가-힣A-Za-z0-9\s]")   # 한글/영문/숫자/공백만 유지
RE_NUMONLY = re.compile(r"^\d+$")
RE_MULTIWS = re.compile(r"\s+")
BAD_TOKENS = {"</s>", "_", "%", "…", "·", "”", "“", "’", "‘", "—", "-", "–"}

def normalize_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = RE_HTML.sub(" ", s)
    s = RE_URL.sub(" ", s)
    s = RE_EMAIL.sub(" ", s)
    s = RE_EMOJI.sub(" ", s)
    s = RE_HANJA.sub(" ", s)
    s = RE_ETC.sub(" ", s)
    s = RE_MULTIWS.sub(" ", s).strip()
    return s

def is_korean_word(w: str) -> bool:
    return bool(re.match(r"^[가-힣]{2,}$", w))

In [3]:
def load_stopwords(path: Path) -> Set[str]:
    if not path.exists():
        print(f"[경고] 불용어 파일이 없습니다: {path}")
        return set()
    with path.open("r", encoding="utf-8") as f:
        raw = [line.strip() for line in f if line.strip() and not line.startswith("#")]
    # 기본형으로 정규화
    normed = set()
    for w in raw:
        # 단어 단독 토큰화 → 기본형 추출
        morphs = okt.pos(w, norm=True, stem=True)
        for token, pos in morphs:
            if len(token) >= 1:
                normed.add(token)
    return normed

custom_stopwords: Set[str] = load_stopwords(STOPWORDS_PATH)
print("불용어 수:", len(custom_stopwords))
list(sorted(list(custom_stopwords)))[:10]

불용어 수: 498


['\t', '가', '가까스로', '가다', '가령', '각', '각각', '각자', '각종', '각하']

In [4]:
def load_korean_sentences_from_transcript(path: Path) -> List[str]:
    sentences: List[str] = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            parts = line.rstrip("\n").split("|")
            if len(parts) > 1:
                ko = parts[1].strip()
                if ko:
                    sentences.append(ko)
    return sentences

raw_sentences: List[str] = load_korean_sentences_from_transcript(TRANSCRIPT_PATH)
print("문장 수:", len(raw_sentences))
raw_sentences[:SHOW_ROWS]

문장 수: 12854


['그는 괜찮은 척하려고 애쓰는 것 같았다.',
 '그녀의 사랑을 얻기 위해 애썼지만 헛수고였다.',
 '용돈을 아껴 써라.',
 '그는 아내를 많이 아낀다.',
 '그 애 전화번호 알아?']

In [5]:
from gensim.models import KeyedVectors

kv = None
if USE_EMBEDDING_FILTER:
    print("임베딩 로드 중…", EMB_PATH)
    kv = KeyedVectors.load_word2vec_format(
        str(EMB_PATH), binary=False, encoding="utf-8", unicode_errors="ignore"
    )
    print("임베딩 단어 수:", len(kv.key_to_index))

임베딩 로드 중… cc.ko.300.vec
임베딩 단어 수: 1999989


In [6]:
def tokenize_ko(
    s: str,
    stopwords: Set[str],
    model: Optional[KeyedVectors] = None,
    min_len: int = MIN_TOKEN_LEN,
    allow_pos: Tuple[str, ...] = ALLOW_POS
) -> List[str]:
    s = normalize_text(s)
    morphs = okt.pos(s, norm=True, stem=True)
    keep: List[str] = []
    for w, p in morphs:
        if w in BAD_TOKENS: continue
        if len(w) < min_len: continue
        if RE_NUMONLY.match(w): continue
        if w in stopwords: continue
        if not p.startswith(allow_pos): continue
        if model is not None and w not in model.key_to_index: continue
        keep.append(w)
    keep = [w for w in keep if is_korean_word(w)]
    return keep

In [7]:
# 토큰화
tokenized_corpus: List[List[str]] = [
    tokenize_ko(s, stopwords=custom_stopwords, model=kv) for s in raw_sentences
]
# 빈/짧은 문장 제거
tokenized_corpus = [t for t in tokenized_corpus if len(t) >= 1]

# 빈도 & 후보 단어 풀
freq = Counter([w for sent in tokenized_corpus for w in sent])
candidate_vocab = [w for w, c in freq.items() if c >= MIN_FREQ and is_korean_word(w)]

print("토큰화된 문장 수:", len(tokenized_corpus))
print("고유 토큰 수:", len(freq))
print("후보 단어 풀 크기:", len(candidate_vocab))

# 미리보기
print("\n[샘플 토큰화 결과]")
for i in range(min(SHOW_ROWS, len(tokenized_corpus))):
    print(f"{i+1}.", tokenized_corpus[i])

print("\n[빈도 상위 20]")
for w, c in freq.most_common(20):
    print(w, c)

print("\n[후보 단어 풀 상위 30]")
candidate_vocab[:30]

토큰화된 문장 수: 11377
고유 토큰 수: 5004
후보 단어 풀 크기: 5004

[샘플 토큰화 결과]
1. ['그녀', '사랑', '수고']
2. ['용돈']
3. ['아내']
4. ['전화번호']
5. ['거기']

[빈도 상위 20]
그녀 238
오늘 216
한국 164
지금 140
정말 131
문제 128
여자 123
요즘 113
어제 112
회사 110
엄마 103
친구 101
아내 100
아주 100
모든 86
아버지 84
남자 78
수가 78
영화 76
마음 76

[후보 단어 풀 상위 30]


['그녀',
 '사랑',
 '수고',
 '용돈',
 '아내',
 '전화번호',
 '거기',
 '시험',
 '감기',
 '사흘',
 '몸살',
 '요즘',
 '공부',
 '장사',
 '아기',
 '안고',
 '엄마',
 '자리',
 '여자',
 '전화',
 '번호',
 '절대',
 '의견',
 '영어',
 '동료',
 '수화',
 '색깔',
 '소영이',
 '얼음',
 '핸드폰']

In [8]:
import random
import numpy as np
from gensim.matutils import unitvec

def cosine(u, v) -> float:
    return float(np.dot(unitvec(u), unitvec(v)))

# 1) similar_pairs_over 보강: a, b 모두 한글만
def similar_pairs_over(vec: KeyedVectors, words, thr: float = 0.8, per_word_topn: int = 50, max_pairs: int = 500):
    # a 후보: 임베딩에 있고 한글만
    words = [w for w in words if (w in vec) and is_korean_word(w)]
    pairs = set()
    random.shuffle(words)

    for a in words:
        try:
            nbrs = vec.most_similar(a, topn=per_word_topn)
        except KeyError:
            continue
        for b, s in nbrs:
            if s < thr or b == a:
                continue
            if (b not in vec) or (not is_korean_word(b)):   # ← b도 한글만
                continue
            u = tuple(sorted((a, b)))
            if u not in pairs:
                pairs.add(u)
                if len(pairs) >= max_pairs:
                    return [(x[0], x[1], cosine(vec[x[0]], vec[x[1]])) for x in pairs]
    return [(x[0], x[1], cosine(vec[x[0]], vec[x[1]])) for x in pairs]

# 2) 기존 pairs가 있으면 한 번 더 걸러주기
try:
    pairs
    pairs = [(a, b, c) for (a, b, c) in pairs if is_korean_word(a) and is_korean_word(b)]
except NameError:
    pass

# 3) 새로 유사쌍 생성
pairs = similar_pairs_over(kv, candidate_vocab, thr=0.8, per_word_topn=50, max_pairs=500)
print("유사도 ≥ 0.8 쌍 개수:", len(pairs))
print(pairs[:10])

# 4) 검증: pairs에 비한글이 남아있는지 확인
nonko_in_pairs = [(a,b) for a,b,_ in pairs if not (is_korean_word(a) and is_korean_word(b))]
print("pairs 내 비한글 쌍 수:", len(nonko_in_pairs))
if nonko_in_pairs:
    print(nonko_in_pairs[:20])

/opt/anaconda3/envs/nlp_env/lib/python3.12/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


유사도 ≥ 0.8 쌍 개수: 26
[('관해', '대해', 0.8129411935806274), ('점점', '점차', 0.8261826038360596), ('적도', '적이', 0.8028584122657776), ('오전', '오후', 0.9102688431739807), ('둘째', '첫째', 0.8528753519058228), ('일요일', '토요일', 0.8150652647018433), ('눈코', '뜰새', 0.8282690048217773), ('감소', '증가', 0.8052759170532227), ('수가', '수는', 0.8367935419082642), ('남자', '여자', 0.8801066279411316)]
pairs 내 비한글 쌍 수: 0


In [9]:
def best_analogy(vec: KeyedVectors, a: str, b: str, c: str, vocab):
    # gensim 방식: positive=[b, c], negative=[a]
    cand = vec.most_similar(positive=[b, c], negative=[a], topn=100)
    for w, s in cand:
        if w not in {a, b, c} and w in vocab and is_korean_word(w):
            return w, float(s)
    # 백업: 직접 계산
    dvec = vec[b] - vec[a] + vec[c]
    sims = []
    for w in vocab:
        if w in {a, b, c}: 
            continue
        sims.append((w, cosine(vec[w], dvec)))
    sims.sort(key=lambda x: x[1], reverse=True)
    return sims[0] if sims else (None, 0.0)

def pick_C(vec: KeyedVectors, a: str, b: str, vocab):
    # A 근방에서 B와 너무 가깝지 않은 후보를 선호
    try:
        nearA = [w for w, s in vec.most_similar(a, topn=100) if w in vocab and w not in {a, b}]
    except KeyError:
        nearA = []
    random.shuffle(nearA)
    for w in nearA:
        try:
            if vec.similarity(w, b) < 0.6:
                return w
        except KeyError:
            continue
    # fallback
    rest = [w for w in vocab if w not in {a, b}]
    return random.choice(rest) if rest else None

def make_round(vec: KeyedVectors, pairs_pool, vocab):
    a, b, _ = random.choice(pairs_pool)
    c = pick_C(vec, a, b, vocab)
    d_pred, conf = best_analogy(vec, a, b, c, vocab)
    return {"A": a, "B": b, "C": c, "D_pred": d_pred, "conf": round(conf, 3)}

In [10]:
def render_round(rnd: dict) -> str:
    return f"관계 [{rnd['A']}: {rnd['B']} = {rnd['C']} : ?]"

def grade_answer(vec: KeyedVectors, rnd: dict, user_answer: str) -> str:
    if rnd["D_pred"] is None or user_answer not in vec:
        sim = 0.0
    else:
        sim = cosine(vec[rnd["D_pred"]], vec[user_answer])
    msg = [
        render_round(rnd),
        f"모델이 예측한 가장 적합한 단어: {rnd['D_pred']}",
        f"당신의 답변과 모델 예측의 유사도: {sim:.2f}",
    ]
    if sim >= 0.7:
        msg.append("정답에 매우 가깝습니다! 🎯")
    elif sim >= 0.4:
        msg.append("오, 거의 근접했어요. 한 번만 더!")
    else:
        msg.append("아쉽네요. 더 생각해보세요.")
    return "\n\n".join(msg)

In [27]:
rnd = make_round(kv, pairs, candidate_vocab)
print(render_round(rnd))

user_answer = input("당신의 답은? ").strip()
print("\n" + grade_answer(kv, rnd, user_answer))

관계 [남자: 여자 = 남성 : ?]

관계 [남자: 여자 = 남성 : ?]

모델이 예측한 가장 적합한 단어: 여성

당신의 답변과 모델 예측의 유사도: 1.00

정답에 매우 가깝습니다! 🎯
